# Judgments and Feature Logging

In this notebook, we cover the first two steps of Learning to Rank. First we grade some documents as relevant/irrelevant for queries, what we call _judgments_. Second, we retrieve some _features_ - metadata about each graded document in our judgments. We call the process of extracting the features from Solr _feature logging_

NOTE: This notebook depends upon TheMovieDB dataset. If you have any issues, please rerun the [Setting up TheMovieDB notebook](1.ch10-setup-TheMovieDB.ipynb)

In [1]:
import requests
import sys
sys.path.append('..')
from aips import *
from ltr.client.solr_client import SolrClient

client = SolrClient(solr_base=solr_url)

## Ommitted from book
A single judgment, grading document 37799 ("The Social Network") as relevant (`grade=1`) for the search query string `social network`

In [2]:
from ltr.judgments import Judgment

Judgment(grade=1, keywords='social network', doc_id=37799)

Judgment(grade=1,qid=1,keywords=social network,doc_id=37799,features=[],weight=1)

## Listing 10.3

A bit bigger judgment list. Here two query strings are graded: `social network` and `star wars`. For `social network` a single movie is graded as relevant, three are irrelevant. Two movies are graded as relevant for `star wars`, three others graded as irrelevant.

In [3]:
mini_judg_list=[
    # for 'social network' query
    Judgment(grade=1, keywords='social network', doc_id='37799'),  # The Social Network
    Judgment(grade=0, keywords='social network', doc_id='267752'), # #chicagoGirl
    Judgment(grade=0, keywords='social network', doc_id='38408'),  # Life As We Know It
    Judgment(grade=0, keywords='social network', doc_id='28303'),  # The Cheyenne Social Club
    
    # for 'star wars' query
    Judgment(grade=1, keywords='star wars', doc_id='11'),     # star wars
    Judgment(grade=1, keywords='star wars', doc_id='1892'),   # return of jedi
    Judgment(grade=0, keywords='star wars', doc_id='54138'),  # Star Trek Into Darkness
    Judgment(grade=0, keywords='star wars', doc_id='85783'),  # The Star
    Judgment(grade=0, keywords='star wars', doc_id='325553'), # Battlestar Galactica
]

mini_judg_list

[Judgment(grade=1,qid=1,keywords=social network,doc_id=37799,features=[],weight=1),
 Judgment(grade=0,qid=1,keywords=social network,doc_id=267752,features=[],weight=1),
 Judgment(grade=0,qid=1,keywords=social network,doc_id=38408,features=[],weight=1),
 Judgment(grade=0,qid=1,keywords=social network,doc_id=28303,features=[],weight=1),
 Judgment(grade=1,qid=2,keywords=star wars,doc_id=11,features=[],weight=1),
 Judgment(grade=1,qid=2,keywords=star wars,doc_id=1892,features=[],weight=1),
 Judgment(grade=0,qid=2,keywords=star wars,doc_id=54138,features=[],weight=1),
 Judgment(grade=0,qid=2,keywords=star wars,doc_id=85783,features=[],weight=1),
 Judgment(grade=0,qid=2,keywords=star wars,doc_id=325553,features=[],weight=1)]

Demonstrating we have no features for any of our judgments.

In [4]:
mini_judg_list[0].features

[]

## Listing 10.4

Create a feature set, the first feature retrieves the relevance score of the search string in the `title` field (hence `title:(${keywords})`), the second feature the same for `overview`, finally the third feature is simply the `release_year` of the movie. 

We create a feature store named `movies` in Solr. We'll use the feature store name as a handle when we want to log features farther down.

In [5]:
requests.delete(f'{solr_url}tmdb/schema/feature-store/movies')

import requests

feature_set = [
    {
      "name" : "title_bm25",
      "store": "movies",
      "class" : "org.apache.solr.ltr.feature.SolrFeature",
      "params" : {
        "q" : "title:(${keywords})"
      }
    },
    {
      "name" : "overview_bm25",
      "store": "movies",
      "class" : "org.apache.solr.ltr.feature.SolrFeature",
      "params" : {
        "q" : "overview:(${keywords})"
      }
    },
    {
      "name" : "release_year",
      "store": "movies",
      "class" : "org.apache.solr.ltr.feature.SolrFeature",
      "params" : {
        "q" : "{!func}release_year"
}}]

resp = requests.put(f'{solr_url}tmdb/schema/feature-store',
                    json=feature_set)
resp.text

'{\n  "responseHeader":{\n    "status":0,\n    "QTime":23}}\n'

# Listing 10.5

Recall we have one relevant and three irrelevant movies for `social network`. Here we retrieve all three features created above for each of the four movies. The special `[features..`], tells Solr to append the features from `movies` feature store using the template param `efi.keywords="social network"` in each document.

In [6]:
import requests

logging_solr_query = {
    "fl": "id,title,[features store=movies efi.keywords=\"social network\"]",
    'q': "id:37799 OR id:267752 OR id:38408 OR id:28303", #social network graded documents
    'rows': 10,
    'wt': 'json'  
}

solr_resp = requests.post(f'{solr_url}tmdb/select', data=logging_solr_query)

solr_resp.json()

{'responseHeader': {'zkConnected': True,
  'status': 0,
  'QTime': 1,
  'params': {'q': 'id:37799 OR id:267752 OR id:38408 OR id:28303',
   'fl': 'id,title,[features store=movies efi.keywords="social network"]',
   'rows': '10',
   'wt': 'json'}},
 'response': {'numFound': 4,
  'start': 0,
  'docs': [{'id': '37799',
    'title': 'The Social Network',
    '[features]': 'title_bm25=8.243603,overview_bm25=3.8143613,release_year=2010.0'},
   {'id': '267752',
    'title': '#chicagoGirl',
    '[features]': 'title_bm25=0.0,overview_bm25=6.0172443,release_year=2013.0'},
   {'id': '28303',
    'title': 'The Cheyenne Social Club',
    '[features]': 'title_bm25=3.4286604,overview_bm25=3.1086721,release_year=1970.0'},
   {'id': '38408',
    'title': 'Life As We Know It',
    '[features]': 'title_bm25=0.0,overview_bm25=4.353118,release_year=2010.0'}]}}

## Omitted from book - parse features Solr response

**The following code is used to generate Listings below. But this parsing code is omitted from the book itself.**

This code simply looks at the solr_resp and populates the corresponding judgments feature vector with the `[features]` from the corresponding solr document.

In [7]:
def populate_features_for_qid(qid, solr_resp, judg_list):
    solr_json = solr_resp.json()
    doc_id_to_features = {}

    # Map Doc Id => Features
    for doc in solr_json['response']['docs']:
        # Parse '[features] array', ie
        # title_bm25=0.0,overview_bm25=13.237938,vote_average=7.0'
        features = doc['[features]']
        features = features.split(',')
        features = [float(ftr.split('=')[1]) for ftr in features]

        doc_id_to_features[doc['id']] = features

    # Save in correct judgment
    for judgment in judg_list:
        if judgment.qid == qid:
            try:
                judgment.features = doc_id_to_features[judgment.doc_id]
            except KeyError:
                pass




## Listing 10.6 (output)

Listing 10.7 is the output of the following, the resulting processing of logging just for `social network`

In [8]:
populate_features_for_qid(qid=1, solr_resp=solr_resp, judg_list=mini_judg_list)
mini_judg_list

[Judgment(grade=1,qid=1,keywords=social network,doc_id=37799,features=[8.243603, 3.8143613, 2010.0],weight=1),
 Judgment(grade=0,qid=1,keywords=social network,doc_id=267752,features=[0.0, 6.0172443, 2013.0],weight=1),
 Judgment(grade=0,qid=1,keywords=social network,doc_id=38408,features=[0.0, 4.353118, 2010.0],weight=1),
 Judgment(grade=0,qid=1,keywords=social network,doc_id=28303,features=[3.4286604, 3.1086721, 1970.0],weight=1),
 Judgment(grade=1,qid=2,keywords=star wars,doc_id=11,features=[],weight=1),
 Judgment(grade=1,qid=2,keywords=star wars,doc_id=1892,features=[],weight=1),
 Judgment(grade=0,qid=2,keywords=star wars,doc_id=54138,features=[],weight=1),
 Judgment(grade=0,qid=2,keywords=star wars,doc_id=85783,features=[],weight=1),
 Judgment(grade=0,qid=2,keywords=star wars,doc_id=325553,features=[],weight=1)]

## Listing 10.7 (output)

Listing 10.8 is the output of the following, which adds features parsed from the `star wars` movies to our judgment list.

In [9]:
logging_solr_query = {
    "fl": "id,title,[features store=movies efi.keywords=\"star wars\"]",
    'q': "id:11 OR id:1892 OR id:54138 OR id:85783 OR id:325553", #star wars graded documents
    'rows': 10,
    'wt': 'json'  
}

resp = requests.post(f'{solr_url}tmdb/select', data=logging_solr_query)

populate_features_for_qid(qid=2, solr_resp=resp, judg_list=mini_judg_list)

mini_judg_list

[Judgment(grade=1,qid=1,keywords=social network,doc_id=37799,features=[8.243603, 3.8143613, 2010.0],weight=1),
 Judgment(grade=0,qid=1,keywords=social network,doc_id=267752,features=[0.0, 6.0172443, 2013.0],weight=1),
 Judgment(grade=0,qid=1,keywords=social network,doc_id=38408,features=[0.0, 4.353118, 2010.0],weight=1),
 Judgment(grade=0,qid=1,keywords=social network,doc_id=28303,features=[3.4286604, 3.1086721, 1970.0],weight=1),
 Judgment(grade=1,qid=2,keywords=star wars,doc_id=11,features=[6.7963624, 0.0, 1977.0],weight=1),
 Judgment(grade=1,qid=2,keywords=star wars,doc_id=1892,features=[0.0, 1.9681965, 1983.0],weight=1),
 Judgment(grade=0,qid=2,keywords=star wars,doc_id=54138,features=[2.444128, 0.0, 2013.0],weight=1),
 Judgment(grade=0,qid=2,keywords=star wars,doc_id=85783,features=[3.1871135, 0.0, 1952.0],weight=1),
 Judgment(grade=0,qid=2,keywords=star wars,doc_id=325553,features=[0.0, 0.0, 2003.0],weight=1)]

## Loading / logging training set (omitted from book)

The following downloads a larger judgment list, parses it, and logs features for each graded document. It just repeats the full logging workflow in this notebook but in one loop.

In [10]:
from ltr.log import FeatureLogger
from ltr.judgments import judgments_open
from itertools import groupby
from ltr import download

ftr_logger=FeatureLogger(client, index='tmdb', feature_set='movies')

with judgments_open('data/ai_pow_search_judgments.txt') as judgment_list:
    for qid, query_judgments in groupby(judgment_list, key=lambda j: j.qid):
        ftr_logger.log_for_qid(judgments=query_judgments, 
                               qid=qid,
                               keywords=judgment_list.keywords(qid))
    
ftr_logger.logged

{!terms f=id}7555,1370,1369,13258,1368,31362,61410,319074,10296,35868,131457,94794,169869,34561,13763,62414,21989,210577,56949,79401,117942,223195,70,22777,43189,318,39829,208982,250761,801,78999,600,26397,45046,271110,59401,79773,321,11553,32221,40082
Searching tmdb [Status: 200]
{!terms f=id}1366,1374,1371,1246,1375,1367,60375,110123,36685,17711,2153,339403,21501,81182,62414,21989,11,12180,140607,1893,330459,152601,17819,198001,170430,18206,47059,18215,70808,351862,154019,70,22777,43189,318,39829,1578,25855,42012,32276,105536
Searching tmdb [Status: 200]
{!terms f=id}14154,365371,116059,335988,354287,390054,346672,369885,293767,381288,860,9614,125842,427760,4247,30335,10207,24964,91893,10951,126148,10428,12699,136698,1557,19433,10431,222461,19855,208988,12180,10253,11,26508,2649,6917,20158,76875,144910
Searching tmdb [Status: 200]
{!terms f=id}9671,9396,9290,248390,221737,13022,10539,17043,78402,25652,36068,77680,6972,182848,259311,293028,64586,40466,104221,183894,3573,12254,17813,98

[Judgment(grade=1,qid=1,keywords=rambo,doc_id=7555,features=[5.9264307, 5.078817, 2008.0],weight=1),
 Judgment(grade=1,qid=1,keywords=rambo,doc_id=1370,features=[5.025649, 5.751174, 1988.0],weight=1),
 Judgment(grade=1,qid=1,keywords=rambo,doc_id=1369,features=[3.4517245, 4.8904457, 1985.0],weight=1),
 Judgment(grade=0,qid=1,keywords=rambo,doc_id=13258,features=[0.0, 4.5890946, 2007.0],weight=1),
 Judgment(grade=1,qid=1,keywords=rambo,doc_id=1368,features=[0.0, 5.394124, 1982.0],weight=1),
 Judgment(grade=0,qid=1,keywords=rambo,doc_id=31362,features=[0.0, 3.7886639, 1988.0],weight=1),
 Judgment(grade=0,qid=1,keywords=rambo,doc_id=61410,features=[0.0, 2.1226306, 2010.0],weight=1),
 Judgment(grade=0,qid=1,keywords=rambo,doc_id=319074,features=[0.0, 0.0, 2015.0],weight=1),
 Judgment(grade=0,qid=1,keywords=rambo,doc_id=10296,features=[0.0, 0.0, 2004.0],weight=1),
 Judgment(grade=0,qid=1,keywords=rambo,doc_id=35868,features=[0.0, 0.0, 2001.0],weight=1),
 Judgment(grade=0,qid=1,keywords=ramb

## Next up, prep data for training

We now have a dataset extracted from the search engine. Next we need to perform some manipulation to the training data. This manipulation turns our slightly strange looking ranking problem into one that looks more like any-other boring machine learning problem.

So when you're ready [please jump into the next section](3.ch10-pairwise-transform.ipynb)